# Задание к лекции "Основы веб-скрапинга и работы с API"

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

## Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [28]:
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup

url = 'https://habr.com/ru/all/'
params = {'python', 'парсинг'}
request = requests.get(url)
soup = BeautifulSoup(request.text, 'lxml')
posts = soup.find_all('article', class_='post') 

In [29]:
habr = pd.DataFrame()
for post in posts:
    post_id = post.parent.attrs.get('id')
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    hubs = post.find_all('a', class_='hub-link') 
    for hub in hubs:
        hub_lower = hub.text.lower()
        if any([hub_lower in desired for desired in params]):
            date = post.find('span', class_='post__time')
            title = post.find('a', class_='post__title_link')
            link = post.find ('a', class_ = 'post__title_link').get('href')
            row = {'date':date, 'title': title, 'link':link}
            habr = pd.concat([habr, pd.DataFrame([row])])
            break
habr

,date,title,link
0,[сегодня в 05:37],[aio api crawler],https://habr.com/ru/post/525912/
